### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
grouped_school = school_data_complete.groupby("school_name")
total_schools = grouped_school.count().shape[0]
#groupby school names so there are only schools, then find the total number of schools by taking the shape of the counts of the dataframe
total_students = school_data_complete.shape[0]
#use the shape of the total data to find every student
total_budget = np.sum(grouped_school.mean()["budget"])
#groupby the schools, take the mean to get one value for each 
avg_math = school_data_complete["math_score"].mean()
avg_read = school_data_complete["reading_score"].mean()
#find the average reading by going through each data point of the original dataframe and taking its mean
passing_math = len(school_data_complete.loc[school_data_complete["math_score"]>=70])/total_students
passing_read = len(school_data_complete.loc[school_data_complete["reading_score"]>=70])/total_students
#finds the total number of students with a grade above or equal to 70, then divides it by total number of students(value is a decimal)
passing_both = len(school_data_complete.loc[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70)])/total_students #ask about the difference between & and "and" because using and here throws an error
#finds the total number of students with both a reading grade and a math grade above or equal to 70 then divides it by total number of students(value is a decimal)

In [6]:
district_summary = {}
district_summary["Total Schools"] = total_schools
district_summary["Total Students"] = total_students
district_summary["Total Budget"] = total_budget
district_summary["Average Math Score"] = avg_math
district_summary["Average Reading Score"] = avg_read
district_summary["% Passing Math"] = f"{passing_math:%}"
district_summary["% Passing Reading"] = f"{passing_read:%}"
district_summary["% Overall Passing"] = f"{passing_both:%}"
district_summary

{'Total Schools': 15,
 'Total Students': 39170,
 'Total Budget': 24649428.0,
 'Average Math Score': 78.98537145774827,
 'Average Reading Score': 81.87784018381414,
 '% Passing Math': '74.980853%',
 '% Passing Reading': '85.805463%',
 '% Overall Passing': '65.172326%'}

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [7]:
school_df = pd.DataFrame(grouped_school.sum())
school_df["Total Students"] = grouped_school.count()["Student ID"]
school_df = pd.merge(school_df, school_data, on= "school_name")
school_df["Total Budget"] = school_df["budget_y"].apply(lambda x: f"${x:,.2f}")
school_df["Per Student Budget"] = school_df["budget_y"] / school_df["Total Students"]
school_df["Average Math Score"] = school_df["math_score"] / school_df["Total Students"]
school_df["Average Reading Score"] = school_df["reading_score"] / school_df["Total Students"]
school_data_complete.loc[school_data_complete["math_score"]>=70].groupby("school_name").count()["Student ID"] 
school_data_complete.loc[school_data_complete["reading_score"]>=70].groupby("school_name").count()
school_data_complete.loc[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70)].groupby("school_name").count()["Student ID"]
school_df = school_df.drop(columns = ["Student ID", "School ID_x", "size_x", "budget_x", "School ID_y", "reading_score","math_score","size_y"])

school_df = school_df.merge(pd.DataFrame(school_data_complete.loc[(school_data_complete["math_score"]>=70)].groupby("school_name").count()["Student ID"].astype(float)).reset_index())
school_df = school_df.rename(columns = {"Student ID" : "% Passing Math"})
school_df["% Passing Math"] = school_df["% Passing Math"] / school_df["Total Students"] * 100
school_df = school_df.merge(pd.DataFrame(school_data_complete.loc[(school_data_complete["reading_score"]>=70)].groupby("school_name").count()["Student ID"].astype(float)).reset_index())
school_df = school_df.rename(columns = {"Student ID" : "% Passing Reading"})
school_df["% Passing Reading"] = school_df["% Passing Reading"] / school_df["Total Students"] * 100
school_df = school_df.merge(pd.DataFrame(school_data_complete.loc[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70)].groupby("school_name").count()["Student ID"].astype(float)).reset_index())
school_df = school_df.rename(columns = {"Student ID" : "Overall % Passing"})
school_df["Overall % Passing"] = school_df["Overall % Passing"] / school_df["Total Students"] *100
school_df = school_df.rename(columns = {"school_name": "School Name", "type":"School Type", "budget_y":"Total Budget"})
school_df = school_df.set_index("School Name")
school_df

,Total Students,School Type,Total Budget,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall % Passing
School Name,,,,,,,,,,
Bailey High School,4976,District,3124928,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,1858,Charter,1081356,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,2949,District,1884411,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,2739,District,1763916,"$1,763,916.00",644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,1468,Charter,917500,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,4635,District,3022020,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,427,Charter,248087,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,2917,District,1910635,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,4761,District,3094650,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [8]:
school_df.sort_values(by = "Overall % Passing", ascending = False).head(5)

,Total Students,School Type,Total Budget,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall % Passing
School Name,,,,,,,,,,
Cabrera High School,1858,Charter,1081356,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,1635,Charter,1043130,"$1,043,130.00",638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,1468,Charter,917500,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,2283,Charter,1319574,"$1,319,574.00",578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,962,Charter,585858,"$585,858.00",609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
school_df.sort_values(by = "Overall % Passing").head(5)

,Total Students,School Type,Total Budget,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall % Passing
School Name,,,,,,,,,,
Rodriguez High School,3999,District,2547363,"$2,547,363.00",637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,2949,District,1884411,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,2917,District,1910635,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,4635,District,3022020,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,4761,District,3094650,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each 
grade level (9th, 10th, 11th, 12th) at each school.
  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
ninth = student_data.loc[student_data["grade"] == "9th"]
tenth = student_data.loc[student_data["grade"] == "10th"]
eleventh = student_data.loc[student_data["grade"] == "11th"]
twelfth = student_data.loc[student_data["grade"] == "12th"]
nine_grouped = ninth.groupby("school_name").mean().reset_index()
ten_grouped = tenth.groupby("school_name").mean().reset_index()
eleven_grouped = eleventh.groupby("school_name").mean().reset_index()
twelfth_grouped = twelfth.groupby("school_name").mean().reset_index()
merge1 = nine_grouped.merge(ten_grouped, on = "school_name", suffixes=["9","10"])
merge2 = merge1.merge(eleven_grouped, on = "school_name")
all_grade_complete = merge2.merge(twelfth_grouped, on = "school_name", suffixes = ["11","12"])

In [ ]:
math_score = all_grade_complete.drop(columns = ["Student ID9","Student ID10","Student ID11","Student ID12","reading_score9","reading_score10","reading_score11","reading_score12"])
math_score = math_score.rename(columns= {"math_score9":"9th", "math_score10":"10th", "math_score11":"11th","math_score12":"12th","school_name":"School Name"})
math_score.set_index("School Name")

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
reading_score = all_grade_complete.drop(columns = ["Student ID9","Student ID10","Student ID11","Student ID12","math_score9","math_score10","math_score11","math_score12"])
reading_score = reading_score.rename(columns= {"reading_score9":"9th", "reading_score10":"10th", "reading_score11":"11th","reading_score12":"12th","school_name":"School Name"})
reading_score.set_index("School Name")

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [22]:
#max,min = np.max(school_df["Per Student Budget"]),np.min(school_df["Per Student Budget"])
#np.arange(min,max,(max-min)//5)
bins = [0, 585, 630, 645, 680]
group_labels = ["<585$", "$585-630", "$630-645", "$645-680"]
school_df["Spending Ranges(Per Student)"] = pd.cut(school_df["Per Student Budget"], bins, labels=group_labels)
school_df.groupby("Spending Ranges(Per Student)").mean().drop(columns = ["Total Students", "Total Budget", "Per Student Budget"])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall % Passing
Spending Ranges(Per Student),,,,,
<585$,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [23]:
bins = [0, 1000, 2000, 5000]
group_labels = ["Small(<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_df["School Size"] = pd.cut(school_df["Total Students"], bins, labels=group_labels)
school_df.groupby("School Size").mean().drop(columns = ["Total Students", "Total Budget", "Per Student Budget"])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall % Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [ ]:
type_df = school_df.groupby("School Type").mean().drop(columns= ["Total Students", "Total Budget", "Per Student Budget"])
type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall % Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


There are two main trends that are relevant. 1. There is a clear distinction between Charter and Distric schools. The overall percentage of indiviudals who pass both math and reading through charter schools is significantly higher than the overall percetnage of indiviudals who pass both math and reading through district schools. The second trend to notice is the lack of trend between the amount of spending per student and the total overall percentage of indivudals who pass both math and reading tests.  